Trabalho Prático 1 - Disciplina de Computação Gráfica – 2024

Dupla:
*   Deise Santana dos Santos
*   Friedrich Naum Espíndola

In [37]:
''' Bibliotecas '''
from lxml import etree

In [ ]:
# Leitura do arquivo XML
tree = etree.parse("entrada.xml")

In [38]:
# Classes
class Window:
    def __init__(self, wmin_x, wmin_y, wmax_x, wmax_y):
        self.wmin_x = wmin_x
        self.wmin_y = wmin_y
        self.wmax_x = wmax_x
        self.wmax_y = wmax_y

class Dot:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Line:
    def __init__(self, dot1, dot2):
        self.dot1 = dot1
        self.dot2 = dot2

class Polygon:
    def __init__(self, dots):
        self.dots = dots

In [39]:
# Busca dos Elementos -> Window
window_xml = tree.find("./window")

wmin_x = window_xml.find("wmin").get("x")
wmin_y = window_xml.find("wmin").get("y")
wmax_x = window_xml.find("wmax").get("x")
wmax_y = window_xml.find("wmax").get("y")

window = Window(wmin_x, wmin_y, wmax_x, wmax_y)

In [40]:
# Busca dos Elementos -> Dots
dots_xml = tree.findall("./ponto")

dots = []

for dot_xml in dots_xml:
  x = dot_xml.attrib['x']
  y = dot_xml.attrib['y']
  dot = Dot(x, y)
  dots.append(dot)

In [41]:
# Busca dos Elementos -> Lines
lines_xml = tree.findall("./reta")

lines = []

for line_xml in lines_xml:
  x1 = line_xml[0].attrib['x']
  y1 = line_xml[0].attrib['y']

  dot1 = Dot(x1, y1)

  x2 = line_xml[1].attrib['x']
  y2 = line_xml[1].attrib['y']

  dot2 = Dot(x2, y2)

  line = Line(dot1, dot2)

  lines.append(line)

In [42]:
# Busca dos Elementos -> Polygon
polygons_xml = tree.findall("./poligono")

polygons = []

for polygon_xml in polygons_xml:
  dots = []

  for dot_xml in polygon_xml.findall("./ponto"):
      x = dot_xml.attrib['x']
      y = dot_xml.attrib['y']
      dot = Dot(x, y)
      dots.append(dot)

  polygons.append(dots)